In [ ]:
import os
import pandas as pd
import json
import re

os.getcwd()

In [ ]:
# 비출판물 말뭉치 중 mscoco로 사용 안 된 자료 가져오기

ms_ko = pd.read_csv('mscoco_kononpup_combined.csv')
ko_non = pd.read_csv('korean_nonpup_lines.csv')
AI_mid = pd.read_json('AIHUB_ko_nonpup_mid.json', orient = 'table', encoding='utf-8')
AI_high = pd.read_json('AIHUB_ko_nonpup_high.json', orient = 'table', encoding='utf-8')
del ms_ko["Unnamed: 0"]
del ko_non["Unnamed: 0"]
rest_ko_non = list(set(ko_non['0']) - set(ms_ko['combined']) - set(AI_mid['ko_nonpup'])-set(AI_high['ko_nonpup']))

In [ ]:
len(rest_ko_non)

In [ ]:
# AIHUB 데이터 가져오기

os.chdir('./AIHUB/label')
target = os.listdir()
target.pop(target.index('.DS_Store'))


image_id = []
questions = []

for name in target:
    path = f'./{name}/하_validate_{name}'
    im = json.load(open(f'{path}/images.json'))
    image = pd.DataFrame(im['images'])

    que = json.load(open(f'{path}/question.json'))
    question = pd.DataFrame(que['questions'])
    
    temp = []
    before = question['image_id'][0]
    
    for i, q in zip(question['image_id'], question['question']):
        if i == before:
            temp.append(q)
        
        else:
            image_id.append(before)
            questions.append("; ".join(temp))
            temp = []
            temp.append(q)
        
        before = i

In [ ]:
df = pd.DataFrame()
df['image_id'] = image_id
df['questions'] = questions

In [ ]:
df[:10]

In [ ]:
from tqdm import tqdm

image_list = []
image_id_list = []

for name in target:
    path = f'./{name}/하_validate_{name}'
    im = json.load(open(f'{path}/images.json'))
    image = pd.DataFrame(im['images'])

    for im, im_id in tqdm(zip(image['image'], image['image_id'])):
        image_list.append(im)
        image_id_list.append(im_id)    

In [ ]:
images = []
for df_im_id in tqdm(df['image_id']):
    for im, im_id in zip(image_list, image_id_list):
        if df_im_id == im_id and im not in images:
            images.append(im)
            break

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()
preprocessed_docs = []

for i in tqdm(df['questions']):
    token_list = []
    for token in mecab.pos(i):
        if token[1] in ['NNG','VA+ETM']:
            token_list.append(token[0])
    preprocessed_docs.append(" ".join(token_list))

In [ ]:
# TF-IDF
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(preprocessed_docs)
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
keywords_AIHUB = []
for i, sent in tqdm(enumerate(preprocessed_docs)):
    temp = []
    for token in sent.split():
        matrix = sp_matrix[i, word2id[token]]
        if 0.3 <= matrix < 1.0:
            temp.append(token)
    if len(temp)==0:
        keywords_AIHUB.append('none')
    else:
        keywords_AIHUB.append(",".join(temp))


In [ ]:
df['keywords'] = keywords_AIHUB

In [ ]:
keywords = list(df['keywords'])
ko_nonpup = []

for words in tqdm(keywords):
    count = 0
    
    for word in words.split(','):
        for sent in rest_ko_non:
            if word in sent and sent not in ko_nonpup:
                ko_nonpup.append(sent)
                count+=1
                break
        if count == 1:
            break
    if count == 0:
        ko_nonpup.append('none')

In [ ]:
print('data: {}, ko_nonpup: {}'.format(len(df), len(ko_nonpup)))

In [ ]:
ko_nonpup.count('none')

In [ ]:
df['ko_nonpup'] = ko_nonpup
df.to_csv('AIHUB_ko_nonpup_low.csv', encoding='utf-8')

In [ ]:
indexes = []
for index, ko in enumerate(df['ko_nonpup']):
    if ko == 'none':
        indexes.append(index)

In [ ]:
df.drop(indexes, inplace=True)

In [ ]:
df.to_json('AIHUB_ko_nonpup_low.json', orient = 'table', force_ascii = False, index =False,indent = 1)